In [1]:
%load_ext autoreload
%autoreload 2
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.tree import plot_tree

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from hummingbird.ml import convert

from random_forest_gemm import DecisionTreeGemm, RandomForestGEMM

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Create RF
X, y = make_classification(n_samples=1000, n_features=4,
                           n_informative=2, n_redundant=0,
                           random_state=0, shuffle=False)
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X, y)

RandomForestClassifier(max_depth=2, random_state=0)

In [4]:
rf_np = RandomForestGEMM(clf, "numpy")
rf_pt = RandomForestGEMM(clf, "torch")
rf_pt_cuda = RandomForestGEMM(clf, "torch", "cuda")

rf_hb = convert(clf, "pytorch")
rf_hb_cuda = convert(clf, "pytorch").to("cuda")

X_pt = torch.Tensor(X)
X_pt_cuda = X_pt.cuda()

In [5]:
%timeit clf.predict(X)

15.7 ms ± 109 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
%timeit rf_np.predict(X)

7.4 ms ± 163 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
%timeit rf_pt.predict(X_pt)

11.5 ms ± 639 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%timeit rf_pt_cuda.predict(X_pt_cuda)

14.5 ms ± 544 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
%timeit rf_hb.predict(X_pt)

2.12 ms ± 181 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
%timeit rf_hb_cuda.predict(X_pt_cuda)

591 µs ± 6.43 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Profiling avec code customisé (cf ``utils.py``)

In [8]:
from utils import profile_command, analyze_stack

In [9]:
pdf = pd.read_pickle("pdf.pickle")

In [10]:
pdf.sort_values("cuda", ascending=False).head()["stack"].values[0]

['/home/jovyan/work/ELTDM/eltdm_project/random_forest_pytorch/random_forest_gemm.py(112): vote',
 '/home/jovyan/work/ELTDM/eltdm_project/random_forest_pytorch/random_forest_gemm.py(115): predict',
 'run_profile.py(26): wrapper',
 '/home/jovyan/work/ELTDM/eltdm_project/random_forest_pytorch/utils.py(70): profile_command',
 'run_profile.py(28): <module>']

In [11]:
pdf.sort_values("cuda", ascending=False).head(n=20)

,name,cpu,cuda,stack
3402,aten::_cat,279.370,2230.718750,[/home/jovyan/work/ELTDM/eltdm_project/random_...
3200,aten::stack,108.535,1444.257812,[/home/jovyan/work/ELTDM/eltdm_project/random_...
1,aten::mm,678.319,775.423973,[/home/jovyan/work/ELTDM/eltdm_project/random_...
2387,aten::eq,47.034,282.207031,[/opt/conda/lib/python3.8/site-packages/torch/...
302,aten::mm,52.412,278.527832,[/home/jovyan/work/ELTDM/eltdm_project/random_...
1011,aten::eq,38.013,277.152344,[/opt/conda/lib/python3.8/site-packages/torch/...
1674,aten::to,30.062,272.734375,[/home/jovyan/work/ELTDM/eltdm_project/random_...
1486,aten::mm,71.220,227.683594,[/home/jovyan/work/ELTDM/eltdm_project/random_...
2305,aten::mm,49.285,216.707031,[/home/jovyan/work/ELTDM/eltdm_project/random_...
1595,aten::mm,42.148,214.652344,[/home/jovyan/work/ELTDM/eltdm_project/random_...


In [12]:
analyze_stack(pdf.sort_values("cuda", ascending=False).head()["stack"].values[0])

,path,line,top,func,content
0,/home/jovyan/work/ELTDM/eltdm_project/random_f...,112,class RandomForestGEMM:,vote,return self.back.stack([e.predict_onehot(X) fo...
1,/home/jovyan/work/ELTDM/eltdm_project/random_f...,115,class RandomForestGEMM:,predict,predictions = self.vote(X)
2,run_profile.py,26,def wrapper():,wrapper,return rf_pt_cuda.predict(X_pt_cuda)
3,/home/jovyan/work/ELTDM/eltdm_project/random_f...,70,def profile_command(func):,profile_command,func()
4,run_profile.py,28,def wrapper():,<module>,pdf = profile_command(wrapper)
